In [1]:
# Librarires
library(dplyr)
library(readr)
library(caret)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: ggplot2


In [2]:
# Feature names
feat_names <- read_delim("./UCIHAR/features.txt", delim = " ", col_names = c("code", "feature"), col_types = "cc")
feat_names <- mutate(feat_names, code=paste0("f",code))
# Number of features
n_feats = dim(feat_names)[1]
n_feats

[1] 561

In [3]:
# Load training data
X_train <- read_table("./UCIHAR/train/X_train.txt", col_names = paste0("f",1:n_feats))
y_train <- read_table("./UCIHAR/train/y_train.txt", col_names = c("label"))

#X_train
#dim(X_train)

Parsed with column specification:
cols(
  .default = col_double()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  label = col_integer()
)


In [4]:
y_train <- read_table("./UCIHAR/train/y_train.txt", col_names = c("label"))

Parsed with column specification:
cols(
  label = col_integer()
)


In [5]:
# Load testing data
X_test <- read_table("./UCIHAR/test/X_test.txt", col_names = paste0("f",1:n_feats))
y_test <- read_table("./UCIHAR/test/y_test.txt", col_names = c("label"))

Parsed with column specification:
cols(
  .default = col_double()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  label = col_integer()
)


In [6]:
# Function for rescaling and centering features
standardize <- function(x){
  (x - mean(x))/sd(x)
}

# Combine train and test (flag by is.train to split back later)
X_train <- mutate(X_train, is.train = 1)
X_test <- mutate(X_test, is.train = 0)
X_full <- rbind(X_train, X_test)


# Standardize (column (n_feats+1) = 652 is `is.train`, which is just a flag)
X_full[,-(n_feats+1)] <- X_full[,-(n_feats+1)] %>% mutate_all(funs(standardize))

#X_full

In [8]:
library(plyr)

In [9]:
library(dplyr)
library(caret)
library(xgboost)
library(MLmetrics)

ERROR: Error in library(xgboost): there is no package called ‘xgboost’


In [12]:

# Split back into train/test
X_tr <- X_full %>% filter(is.train == 1) %>% dplyr::select(-is.train)
X_tst <- X_full %>% filter(is.train == 0) %>% dplyr::select(-is.train)
# XGboost wants levels to start with 0
y_tr <- as.factor(y_train$label)
y_tr <- revalue(y_tr, c('6'='5', '5'='4', '4'='3', '3'='2', '2'='1', '1'='0'))
y_tr <- as.numeric(levels(y_tr))[y_tr]

# XGB style matrices
dtrain <- xgb.DMatrix(as.matrix(X_tr), label = y_tr)
watchlist <- list(train=dtrain)


ERROR: Error in xgb.DMatrix(as.matrix(X_tr), label = y_tr): could not find function "xgb.DMatrix"


In [17]:
grid_1 <- expand.grid(eta = c(0.3,0.5,0.7),
                      max_depth = c(1,2,6,8))
cv.results <- data.frame(grid_1) %>% mutate(nrounds = 0) %>% mutate(mll_train = 0) %>% mutate(mll_test = 0)

for (ind in 1:nrow(cv.results)){
    print(ind)
}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
